<a href="https://colab.research.google.com/github/Guo-bot-1998/Appendicitis/blob/master/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nibabel as nib
import numpy as np
import os
import pandas as pd
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
from helper import  read_data
from helper import  read_label
from helper import isgpu


device = isgpu()

GPU is available


處理資料

In [2]:
cd D:\Program\data\AOCR2024\

D:\Program\data\AOCR2024


In [3]:
data = read_data("TrainValid_Image\\train_data", termi=2)
df = read_label("TrainValid_ground_truth.csv")
df.set_index('id', inplace=True)
# df
processed = []
for key, value in data.items():
  scan  = df.loc[df.index.str.startswith(key+'_')]
  labels = np.array(scan['label'])
  processed.append((value, labels))
len(processed)

2

In [4]:
# 轉換每個掃描中的圖像和標籤
images_list = []
labels_list = []

for value, label in processed:
    # value.shape 為 (512, 512, n)，label.shape 為 (n,)
    for i in range(value.shape[2]):
        image = value[:, :, i]
        image_tensor = torch.from_numpy(image).float().unsqueeze(0)  # 添加通道維度
        label_tensor = torch.tensor(label[i], dtype=torch.float32)
        images_list.append(image_tensor)
        labels_list.append(label_tensor)

# 合併成批次數據
images = torch.stack(images_list)
labels = torch.stack(labels_list)


訓練

In [5]:
num_epochs = 10
batch_size = 32
lr = 0.001
num_batches = len(images) // batch_size

import models.Unet

# 初始化模型、損失函數和優化器
model = models.Unet.FC(in_channels=1).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr)



# 訓練循環
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i in range(num_batches):
        batch_images = images[i*batch_size:(i+1)*batch_size].to(device)
        batch_labels = labels[i*batch_size:(i+1)*batch_size].to(device)

        optimizer.zero_grad()
        outputs = model(batch_images)
        outputs = outputs.squeeze()
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/num_batches}")


Epoch 1, Loss: 2.3039690017700196
Epoch 2, Loss: 5.625
Epoch 3, Loss: 5.625
Epoch 4, Loss: 5.625
Epoch 5, Loss: 5.625
Epoch 6, Loss: 5.625
Epoch 7, Loss: 5.625
Epoch 8, Loss: 5.625
Epoch 9, Loss: 5.625
Epoch 10, Loss: 5.625


儲存模型參數

In [6]:
cd D:\Program\machineLearning\Appendicitis\

D:\Program\machineLearning\Appendicitis


In [7]:
import json
appendix = input("請輸入要儲存的編號:")
modelname = model.__class__.__name__
filename = f"params/{modelname}/{modelname}{appendix}"
if os.path.isfile(filename+'.pth'):
    print(f"{filename}.pth exist.")
else:
    torch.save(model.state_dict(), f'{filename}.pth')

params = {
    'num_epochs': num_epochs,
    'batch_size': batch_size,
    'learning_rate': lr
}

if os.path.isfile(filename+'.json'):
    print(f"{filename}.json exist.")
else:
    with open(f'{filename}.json', 'w') as f:
        json.dump(params, f)

params/FC/FC.json exist.


True Postive / Total Postive
F1